This notebook runs the event segmentation analyses.

## Import libraries

In [12]:
import numpy as np
import pandas as pd
import brainiak.eventseg.event
import hypertools as hyp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
import pickle

def score_model(mcorr, model, k, s):
    i1, i2 = np.where(np.round(model.segments_[0])==1)
    w = np.zeros_like(model.segments_[0])
    w[i1,i2] = 1
    w = np.dot(w, w.T).astype(bool)
    return mcorr[w].mean()/mcorr[~w].mean() - k/s
    
def reduce_model(m, ev):
    """Reduce a model based on event labels"""
    w = (np.round(ev.segments_[0])==1).astype(bool)
    return np.array([m[wi, :].mean(0) for wi in w.T])

%matplotlib inline

## Setting paths

In [7]:
datadir = '../../data/processed/'

## Load data

In [9]:
video_model, recall_models = np.load(datadir+'models_t100_v50_r10.npy')

In [16]:
ks = list(range(2, 50))
maxk = []
mcorr = np.corrcoef(video_model)
s = 10
cs = []

for k in ks:
    
    # fit model
    ev = brainiak.eventseg.event.EventSegment(k)
    ev.fit(video_model)
    
    # score model
    c = score_model(mcorr, ev, k, s)
    cs.append(c)
    print(k, c)
    
print(ks[np.argmax(cs)])

2 -10.128918054405212
3 -17.00797935606088
4 -22.527770602626493
5 -37.047163823757685
6 -52.432859456617436
7 -59.15419823934286
8 -86.04022228160703
9 -133.36130785912854
10 -198.28786864314063
11 -295.170031770841
12 -5489.809080387881
13 2863.52260266772
14 279.9076678506955
15 189.32035211796736
16 155.6611987504374
17 131.80299966583988
18 115.39873831147254
19 103.18510498831188
20 93.66781576702915
21 88.80190057306737
22 82.19720938923747
23 74.12309240457814
24 69.14241210535243
25 69.81938950698537
26 65.02740569926117


KeyboardInterrupt: 

## Fit event segmentation model to video

In [12]:
ev = brainiak.eventseg.event.EventSegment(34)
ev.fit(video_model)
video_events = reduce_model(video_model, ev)

## Save models

In [5]:
# with open('../data/video_eventseg_model', 'wb') as f:
#     pickle.dump(ev, f)
# np.save('../data/video_events', video_events)

## Get video event times

In [5]:
video_event_times = []
for s in ev.segments_[0].T:
    tp = np.where(np.round(s)==1)[0]
    video_event_times.append((tp[0], tp[-1]))
np.save(datadir+'video_event_times', video_event_times)

## Fit event segmentation model to recall

In [14]:
# ks = list(range(2, 30))
# maxk = []
# for i, sub in enumerate(recall_models):
#     mcorr = np.corrcoef(sub)
#     cs = []
#     for k in ks:
#         ev = brainiak.eventseg.event.EventSegment(k)
#         ev.fit(sub)
#         i1, i2 = np.where(np.round(ev.segments_[0])==1)
#         w = np.zeros_like(ev.segments_[0])
#         w[i1,i2] = 1
#         w = np.dot(w, w.T).astype(bool)
#         c = mcorr[w].mean()/mcorr[~w].mean() - k/50
#         cs.append(c)
#     m = ks[np.argmax(cs)]
#     maxk.append(m)
#     print(i, m)
maxk = np.array([8, 14, 11, 9, 14, 10, 17, 17, 10, 20, 19, 25, 27, 14, 10, 14, 23])

0 8
1 14
2 11
3 9
4 14
5 10
6 17
7 17
8 10
9 20
10 19
11 25
12 27
13 14
14 10
15 14
16 23


## Fit model to recall using best k

In [19]:
recall_events = []
recall_event_times = []
recall_eventseg_models = []
for i, k in enumerate(maxk):
    ev = brainiak.eventseg.event.EventSegment(k)
    ev.fit(recall_models[i])
    m = reduce_model(recall_models[i], ev)
    recall_events.append(m)
    recall_times = []
    for s in ev.segments_[0].T:
        tp = np.where(np.round(s)==1)[0]
        recall_times.append((tp[0], tp[-1]))
    recall_event_times.append(recall_times)
    recall_eventseg_models.append(ev)

## Create average recall model

In [20]:
matches = np.array([np.argmax(1 - cdist(video_events, r, 'correlation'), 0) for r in recall_events])
avg_recalls = [[] for _ in video_events]
for match, r in zip(matches, recall_events):
    for i, m in enumerate(match):
        avg_recalls[m].append(r[i,:])
avg_recall_events = np.array(list(map(lambda r: np.mean(r, 0) if len(r)>0 else np.zeros((100,)), avg_recalls)))
# avg_recall_events = np.array([a.reshape(100,) for a in avg_recall_events if a.shape==(100,)])

## Create 2D embeddings

In [21]:
np.random.seed(10)
embeddings = hyp.reduce(recall_events+[video_events]+[avg_recall_events], reduce='UMAP', ndims=2)

## Save models

In [11]:
# np.save(datadir+'avg_recall_events', avg_recall_events)
# np.save(datadir+'embeddings', [embeddings[:-2], embeddings[-2], embeddings[:-1]])
# np.save(datadir+'labels', matches)
# np.save(datadir+'recall_events', recall_events)
# np.save(datadir+'recall_event_times', recall_event_times)
# with open(datadir+'recall_eventseg_models', 'wb') as f:
#     pickle.dump(recall_eventseg_models, f)